In [24]:
import pandas as pd
import xml.etree.ElementTree as ET
import geopandas
import numpy as np
from shapely.geometry import Point
import json
import copy

In [25]:
ET.register_namespace('', 'http://www.w3schools.com')
namespace = {'xmlns': 'http://www.w3schools.com'}
tree = ET.parse('data/sjobo_initial_solution.xml')

In [26]:
with open("data/population_general_divided.json") as f:
    j = json.load(f)

In [27]:
j.keys()

dict_keys(['population_within_pt', 'population_within_other', 'population_in_pt', 'population_out_pt', 'population_in_drtable', 'population_out_drtable', 'population_in_other', 'population_out_other'])

In [28]:
within_sample = pd.DataFrame(j['population_within_other']).sample(int(len(j['population_within_other'])))
within_sample['direction'] = 'within'
in_sample = pd.DataFrame(j['population_in_drtable']).sample(int(len(j['population_in_drtable'])))
in_sample['direction'] = 'in'
out_sample = pd.DataFrame(j['population_out_drtable']).sample(int(len(j['population_out_drtable'])))
out_sample['direction'] = 'out'

In [29]:
personsdf = pd.concat([within_sample, in_sample, out_sample])

In [30]:
persons = []
for person in personsdf.itertuples():
    slat = person.activities[0]['coord']['lat']
    slon = person.activities[0]['coord']['lon']
    elat = person.activities[1]['coord']['lat']
    elon = person.activities[1]['coord']['lon']
    
    attr = copy.copy(person.attributes)
    attr.update({'direction': person.direction})
    
    persons.append({
        'id': person.id,
        'start_longitude': slon,
        'start_latitude': slat,
        'end_longitude': elon,
        'end_latitude': elat,
        'type': person.activities[1]['type'],
        'time': person.activities[1]['start_time'],
        'szone': person.activities[0]['zone'],
        'ezone': person.activities[1]['zone'],
        'attributes': attr,
    })
persons = pd.DataFrame(persons)

In [31]:
persons

,id,start_longitude,start_latitude,end_longitude,end_latitude,type,time,szone,ezone,attributes
0,2290720,13.709747,55.634970,13.690175,55.631357,12,38880,1265,1265,"{'age': 'Ald65_W', 'direction': 'within'}"
1,2292942,13.705721,55.638299,13.722546,55.630767,4,67620,1265,1265,"{'age': 'Ald7_15', 'direction': 'within'}"
2,2285469,13.696057,55.632325,13.708125,55.633613,4,21240,1265,1265,"{'age': 'Ald7_15', 'direction': 'within'}"
3,3136944,13.734054,55.595497,13.718121,55.590754,6,22380,1265,1265,"{'age': 'Ald65_W', 'direction': 'within'}"
4,2280543,13.683168,55.643395,13.691851,55.629112,12,41580,1265,1265,"{'age': 'Ald45_64', 'direction': 'within'}"
...,...,...,...,...,...,...,...,...,...,...
51033,2166551,13.790154,55.706001,13.419737,55.649815,6,53580,1265,1281,"{'age': 'Ald25_44', 'direction': 'out'}"
51034,702786,13.688529,55.634295,13.014194,55.721222,1,56520,1265,1262,"{'age': 'Ald25_44', 'direction': 'out'}"
51035,2443356,13.601053,55.579762,13.502041,55.629082,4,29940,1265,1281,"{'age': 'Ald7_15', 'direction': 'out'}"
51036,1749846,13.708580,55.628491,12.978027,55.580587,1,54420,1265,1280,"{'age': 'Ald25_44', 'direction': 'out'}"


In [32]:
persons.to_excel('data/sjöbo_trips_others_sep_direction.xlsx', index=False)

# run "IVT between points with OSRM" 

In [33]:
direct_time = pd.read_excel('data/direct_time.xlsx')

In [34]:
persons['direct_time'] = direct_time.direct_time

In [35]:
persons

,id,start_longitude,start_latitude,end_longitude,end_latitude,type,time,szone,ezone,attributes,direct_time
0,2290720,13.709747,55.634970,13.690175,55.631357,12,38880,1265,1265,"{'age': 'Ald65_W', 'direction': 'within'}",993.4
1,2292942,13.705721,55.638299,13.722546,55.630767,4,67620,1265,1265,"{'age': 'Ald7_15', 'direction': 'within'}",62.7
2,2285469,13.696057,55.632325,13.708125,55.633613,4,21240,1265,1265,"{'age': 'Ald7_15', 'direction': 'within'}",169.9
3,3136944,13.734054,55.595497,13.718121,55.590754,6,22380,1265,1265,"{'age': 'Ald65_W', 'direction': 'within'}",282.6
4,2280543,13.683168,55.643395,13.691851,55.629112,12,41580,1265,1265,"{'age': 'Ald45_64', 'direction': 'within'}",1098.1
...,...,...,...,...,...,...,...,...,...,...,...
51033,2166551,13.790154,55.706001,13.419737,55.649815,6,53580,1265,1281,"{'age': 'Ald25_44', 'direction': 'out'}",781.9
51034,702786,13.688529,55.634295,13.014194,55.721222,1,56520,1265,1262,"{'age': 'Ald25_44', 'direction': 'out'}",2912.0
51035,2443356,13.601053,55.579762,13.502041,55.629082,4,29940,1265,1281,"{'age': 'Ald7_15', 'direction': 'out'}",1201.5
51036,1749846,13.708580,55.628491,12.978027,55.580587,1,54420,1265,1280,"{'age': 'Ald25_44', 'direction': 'out'}",717.1


In [36]:
max_ivt = []
for person in persons.itertuples():
#     if person.szone == 1265 and person.ezone == 1265:
#         max_ivt.append(person.direct_time * 1.7)
#     elif person.szone == 1265 and person.ezone != 1265:
#         max_ivt.append(person.direct_time * 1.95)
#     elif person.szone != 1265 and person.ezone == 1265:
#         max_ivt.append(person.direct_time * 1.55)
    max_ivt.append(person.direct_time * 1.5)

In [37]:
persons['max_ivt'] = max_ivt

In [38]:
persons

,id,start_longitude,start_latitude,end_longitude,end_latitude,type,time,szone,ezone,attributes,direct_time,max_ivt
0,2290720,13.709747,55.634970,13.690175,55.631357,12,38880,1265,1265,"{'age': 'Ald65_W', 'direction': 'within'}",993.4,1490.10
1,2292942,13.705721,55.638299,13.722546,55.630767,4,67620,1265,1265,"{'age': 'Ald7_15', 'direction': 'within'}",62.7,94.05
2,2285469,13.696057,55.632325,13.708125,55.633613,4,21240,1265,1265,"{'age': 'Ald7_15', 'direction': 'within'}",169.9,254.85
3,3136944,13.734054,55.595497,13.718121,55.590754,6,22380,1265,1265,"{'age': 'Ald65_W', 'direction': 'within'}",282.6,423.90
4,2280543,13.683168,55.643395,13.691851,55.629112,12,41580,1265,1265,"{'age': 'Ald45_64', 'direction': 'within'}",1098.1,1647.15
...,...,...,...,...,...,...,...,...,...,...,...,...
51033,2166551,13.790154,55.706001,13.419737,55.649815,6,53580,1265,1281,"{'age': 'Ald25_44', 'direction': 'out'}",781.9,1172.85
51034,702786,13.688529,55.634295,13.014194,55.721222,1,56520,1265,1262,"{'age': 'Ald25_44', 'direction': 'out'}",2912.0,4368.00
51035,2443356,13.601053,55.579762,13.502041,55.629082,4,29940,1265,1281,"{'age': 'Ald7_15', 'direction': 'out'}",1201.5,1802.25
51036,1749846,13.708580,55.628491,12.978027,55.580587,1,54420,1265,1280,"{'age': 'Ald25_44', 'direction': 'out'}",717.1,1075.65


In [39]:
persons.to_excel('data/sjöbo_trips_others_directions_51000.xlsx', index=False)

In [40]:
# persons.replace(0, np.NaN).dropna().sample(1500).to_excel('data/sjöbo_trips_others_sample1500.xlsx', index=False)

In [41]:
pop = []
pop_drt_transit = []
for person in persons.itertuples():
    
    p = {'activities': [
                        {"coord": {"lat": person.start_latitude, "lon": person.start_longitude},
                         "start_time": 0,
                         "end_time": person.time,
                         "type": "HOME",
                         "zone": int(person.szone)
                        },
                        {"coord": {"lat": person.end_latitude, "lon": person.end_longitude},
                         "start_time": int(person.time),
                         "end_time": 86400,
                         "type": "WORK",
                         "zone": person.ezone
                        }
                        ],
         "id": person.id,
         'attributes': person.attributes
        }
    pop.append(p)
    if person.attributes['direction'] in ['in', 'out']:
        pop_drt_transit.append(p)

j = {"persons": pop}
j2 = {"persons": pop_drt_transit}

In [78]:
with open('data/sjöbo_trips_others_directions_51000.json', 'w') as f:
    json.dump(j, f)
with open('data/sjöbo_trips_others_directions_51000_DRT_TRANSIT_only.json', 'w') as f:
    json.dump(j2, f)    

# ^That json file should go to DRT Sim. Branch: "sjöbo other demand experiment generate DRT TRANSIT".

# take trip dump from there

In [42]:
persons.index = persons.id

In [43]:
persons

,id,start_longitude,start_latitude,end_longitude,end_latitude,type,time,szone,ezone,attributes,direct_time,max_ivt
id,,,,,,,,,,,,
2290720,2290720,13.709747,55.634970,13.690175,55.631357,12,38880,1265,1265,"{'age': 'Ald65_W', 'direction': 'within'}",993.4,1490.10
2292942,2292942,13.705721,55.638299,13.722546,55.630767,4,67620,1265,1265,"{'age': 'Ald7_15', 'direction': 'within'}",62.7,94.05
2285469,2285469,13.696057,55.632325,13.708125,55.633613,4,21240,1265,1265,"{'age': 'Ald7_15', 'direction': 'within'}",169.9,254.85
3136944,3136944,13.734054,55.595497,13.718121,55.590754,6,22380,1265,1265,"{'age': 'Ald65_W', 'direction': 'within'}",282.6,423.90
2280543,2280543,13.683168,55.643395,13.691851,55.629112,12,41580,1265,1265,"{'age': 'Ald45_64', 'direction': 'within'}",1098.1,1647.15
...,...,...,...,...,...,...,...,...,...,...,...,...
2166551,2166551,13.790154,55.706001,13.419737,55.649815,6,53580,1265,1281,"{'age': 'Ald25_44', 'direction': 'out'}",781.9,1172.85
702786,702786,13.688529,55.634295,13.014194,55.721222,1,56520,1265,1262,"{'age': 'Ald25_44', 'direction': 'out'}",2912.0,4368.00
2443356,2443356,13.601053,55.579762,13.502041,55.629082,4,29940,1265,1281,"{'age': 'Ald7_15', 'direction': 'out'}",1201.5,1802.25


In [44]:
persons['drt_tw_start_left'] = None
persons['drt_tw_start_right'] = None
persons['drt_tw_end_left'] = None
persons['drt_tw_end_right'] = None
persons['max_drt_duration'] = None

In [45]:
with open('data/trip_dump_DRT_TRANSIT.json') as json_file:
    data = json.load(json_file)
    
for person in data['person']:
    planned_trip = person['planned_trips']
    if len(planned_trip) == 0:
        continue
    
    planned_trip = planned_trip[0]

    main_mode = planned_trip['main_mode']

    if main_mode != 'DRT_TRANSIT':
        continue

    legs = planned_trip['legs']
    drt_leg = [leg for leg in legs if leg['mode'] == 'DRT'][0]
    pid = person['id']
    
    persons.loc[[pid], ['start_longitude']] = drt_leg['start_coord']['lon']
    persons.loc[[pid], ['start_latitude']] = drt_leg['start_coord']['lat']
    
    persons.loc[[pid], ['end_longitude']] = drt_leg['end_coord']['lon']
    persons.loc[[pid], ['end_latitude']] = drt_leg['end_coord']['lat']
    
    persons.loc[[pid], ['drt_tw_start_left']] = planned_trip['drt_tw_start_left']
    persons.loc[[pid], ['drt_tw_start_right']] = planned_trip['drt_tw_start_right']
    
    persons.loc[[pid], ['drt_tw_end_left']] = planned_trip['drt_tw_end_left']
    persons.loc[[pid], ['drt_tw_end_right']] = planned_trip['drt_tw_end_right']
    
    persons.loc[[pid], ['max_drt_duration']] = planned_trip['max_drt_duration']

In [48]:
persons[persons['max_drt_duration'] > 0]

,id,start_longitude,start_latitude,end_longitude,end_latitude,type,time,szone,ezone,attributes,direct_time,max_ivt,drt_tw_start_left,drt_tw_start_right,drt_tw_end_left,drt_tw_end_right,max_drt_duration
id,,,,,,,,,,,,,,,,,
2590157,2590157,14.026305,55.652168,13.891297,55.655108,4,37200,1290,1265,"{'age': 'Ald20_24', 'direction': 'in'}",2985.7,4478.55,40961.0,42013.45,40961.0,42013.45,1052.45
2948372,2948372,13.928073,55.565458,13.913162,55.609397,5,19680,1270,1265,"{'age': 'Ald25_44', 'direction': 'in'}",2667.8,4001.70,19375.0,20182.4,19375.0,20182.4,807.4
1431507,1431507,13.702021,55.630894,13.722546,55.630767,4,43020,1281,1265,"{'age': 'Ald20_24', 'direction': 'in'}",1900.4,2850.60,44395.0,44916.2,44395.0,44916.2,521.2
2303108,2303108,13.704680,55.615844,13.701003,55.645659,2,23880,1286,1265,"{'age': 'Ald45_64', 'direction': 'in'}",2473.1,3709.65,24562.0,25812.1,24562.0,25812.1,1250.1
701294,701294,13.655586,55.637281,13.689296,55.642036,2,24780,1280,1265,"{'age': 'Ald45_64', 'direction': 'in'}",2006.1,3009.15,27277.0,28255.2,27277.0,28255.2,978.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288404,2288404,13.707041,55.642566,13.774438,55.612733,4,49380,1265,1270,"{'age': 'Ald16_19', 'direction': 'out'}",2854.3,4281.45,47511.3,48235.0,47511.3,48235.0,723.7
2201374,2201374,13.715640,55.639668,13.694162,55.632336,1,75120,1265,1281,"{'age': 'Ald25_44', 'direction': 'out'}",2572.2,3858.30,74167.2,74643.0,74167.2,74643.0,475.8
3127296,3127296,13.695455,55.580444,13.774438,55.612733,2,56400,1265,1270,"{'age': 'Ald45_64', 'direction': 'out'}",4376.3,6564.45,54551.85,55435.0,54551.85,55435.0,883.15


In [49]:
persons_f = persons[((persons.szone == 1265) | (persons.ezone == 1265)) & (persons.direct_time > 120)]

In [50]:
persons_f

,id,start_longitude,start_latitude,end_longitude,end_latitude,type,time,szone,ezone,attributes,direct_time,max_ivt,drt_tw_start_left,drt_tw_start_right,drt_tw_end_left,drt_tw_end_right,max_drt_duration
id,,,,,,,,,,,,,,,,,
2290720,2290720,13.709747,55.634970,13.690175,55.631357,12,38880,1265,1265,"{'age': 'Ald65_W', 'direction': 'within'}",993.4,1490.10,None,None,None,None,None
2285469,2285469,13.696057,55.632325,13.708125,55.633613,4,21240,1265,1265,"{'age': 'Ald7_15', 'direction': 'within'}",169.9,254.85,None,None,None,None,None
3136944,3136944,13.734054,55.595497,13.718121,55.590754,6,22380,1265,1265,"{'age': 'Ald65_W', 'direction': 'within'}",282.6,423.90,None,None,None,None,None
2280543,2280543,13.683168,55.643395,13.691851,55.629112,12,41580,1265,1265,"{'age': 'Ald45_64', 'direction': 'within'}",1098.1,1647.15,None,None,None,None,None
2284230,2284230,13.705333,55.633000,13.719270,55.636871,12,54120,1265,1265,"{'age': 'Ald25_44', 'direction': 'within'}",529.9,794.85,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2166551,2166551,13.790154,55.706001,13.651962,55.637622,6,53580,1265,1281,"{'age': 'Ald25_44', 'direction': 'out'}",781.9,1172.85,52195.75,53266.0,52195.75,53266.0,1070.25
702786,702786,13.688529,55.634295,13.014194,55.721222,1,56520,1265,1262,"{'age': 'Ald25_44', 'direction': 'out'}",2912.0,4368.00,None,None,None,None,None
2443356,2443356,13.601053,55.579762,13.502041,55.629082,4,29940,1265,1281,"{'age': 'Ald7_15', 'direction': 'out'}",1201.5,1802.25,None,None,None,None,None


In [51]:
persons_f[persons_f['max_drt_duration'] > 0]

,id,start_longitude,start_latitude,end_longitude,end_latitude,type,time,szone,ezone,attributes,direct_time,max_ivt,drt_tw_start_left,drt_tw_start_right,drt_tw_end_left,drt_tw_end_right,max_drt_duration
id,,,,,,,,,,,,,,,,,
2590157,2590157,14.026305,55.652168,13.891297,55.655108,4,37200,1290,1265,"{'age': 'Ald20_24', 'direction': 'in'}",2985.7,4478.55,40961.0,42013.45,40961.0,42013.45,1052.45
2948372,2948372,13.928073,55.565458,13.913162,55.609397,5,19680,1270,1265,"{'age': 'Ald25_44', 'direction': 'in'}",2667.8,4001.70,19375.0,20182.4,19375.0,20182.4,807.4
1431507,1431507,13.702021,55.630894,13.722546,55.630767,4,43020,1281,1265,"{'age': 'Ald20_24', 'direction': 'in'}",1900.4,2850.60,44395.0,44916.2,44395.0,44916.2,521.2
2303108,2303108,13.704680,55.615844,13.701003,55.645659,2,23880,1286,1265,"{'age': 'Ald45_64', 'direction': 'in'}",2473.1,3709.65,24562.0,25812.1,24562.0,25812.1,1250.1
701294,701294,13.655586,55.637281,13.689296,55.642036,2,24780,1280,1265,"{'age': 'Ald45_64', 'direction': 'in'}",2006.1,3009.15,27277.0,28255.2,27277.0,28255.2,978.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288404,2288404,13.707041,55.642566,13.774438,55.612733,4,49380,1265,1270,"{'age': 'Ald16_19', 'direction': 'out'}",2854.3,4281.45,47511.3,48235.0,47511.3,48235.0,723.7
2201374,2201374,13.715640,55.639668,13.694162,55.632336,1,75120,1265,1281,"{'age': 'Ald25_44', 'direction': 'out'}",2572.2,3858.30,74167.2,74643.0,74167.2,74643.0,475.8
3127296,3127296,13.695455,55.580444,13.774438,55.612733,2,56400,1265,1270,"{'age': 'Ald45_64', 'direction': 'out'}",4376.3,6564.45,54551.85,55435.0,54551.85,55435.0,883.15


In [ ]:
persons_f.sample()

In [75]:
persons.to_excel('data/sjöbo_trips_others_directions_51000.xlsx')